In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data Science /Penambangan Data (Data Mining)/Unsupervised Learning/Associan Rules/Tugas 4 Associan Rules/Data1.csv')
data.head()

,Timestamp,Nama,Minuman kekinian apa saja yang sering Anda konsumsi?
0,12/26/2025 20:53:10,anggit,"Kopi Susu, Thai Tea, Matcha latte, Kopi gula a..."
1,12/26/2025 20:53:20,Aszra,"Kopi Susu, Thai Tea, Kopi gula aren"
2,12/26/2025 20:54:20,Isthi,Yakult drink
3,12/26/2025 20:56:03,Akbar,"Kopi Susu, Yakult drink, Chocolate latte"
4,12/26/2025 20:57:58,Yaqin,"Kopi Susu, Yakult drink"


In [ ]:
print(data.columns)

Index(['Timestamp', 'Nama',
       'Minuman kekinian apa saja yang sering Anda konsumsi?'],
      dtype='object')


# **Pre-Processing**

**Pengambilan Kolom Minuman**

In [ ]:
kolom_minuman ='Minuman kekinian apa saja yang sering Anda konsumsi?'
data_minuman = data[kolom_minuman]

*Karena fungsi association rule bertujuan untuk menemukan pola keterkaitan dan rekomendasi antar barang berdasarkan kemunculan item dalam satu transaksi, maka atribut yang digunakan dalam analisis ini dibatasi pada kolom minuman sebagai representasi item yang dikonsumsi oleh responden.*

In [ ]:
data_minuman.head()

,Minuman kekinian apa saja yang sering Anda konsumsi?
0,"Kopi Susu, Thai Tea, Matcha latte, Kopi gula a..."
1,"Kopi Susu, Thai Tea, Kopi gula aren"
2,Yakult drink
3,"Kopi Susu, Yakult drink, Chocolate latte"
4,"Kopi Susu, Yakult drink"


**Split Data**

In [ ]:
data_minuman = (data_minuman
                .str.split(',')
)

*Karena data minuman awalnya berbentuk string mentah dengan pemisah berupa tanda koma (,), maka diperlukan proses pemisahan (split) untuk mengubah setiap respons menjadi kumpulan item per transaksi. Tahapan ini dilakukan agar data dapat ditransformasikan ke dalam bentuk numerik melalui proses encoding sebagai dasar analisis association rules.*

In [ ]:
data_minuman.head()

,Minuman kekinian apa saja yang sering Anda konsumsi?
0,"[Kopi Susu, Thai Tea, Matcha latte, Kopi gu..."
1,"[Kopi Susu, Thai Tea, Kopi gula aren]"
2,[Yakult drink]
3,"[Kopi Susu, Yakult drink, Chocolate latte]"
4,"[Kopi Susu, Yakult drink]"


In [ ]:
data_minuman_clean = [
    [item.strip() for item in transaksi]
    for transaksi in data_minuman
]

*Karena data yang digunakan dalam penelitian ini diperoleh dari hasil kuesioner, terdapat ketidakkonsistenan pada penulisan data responden, khususnya perbedaan penggunaan spasi dan format penamaan item minuman.*

**Encoding**

In [ ]:
te = TransactionEncoder()
te_array = te.fit(data_minuman_clean).transform(data_minuman_clean)

df_encoded = pd.DataFrame(te_array, columns=te.columns_)

*Proses encoding dilakukan untuk mentransformasikan data kategorikal ke dalam bentuk numerik (integer/biner) sehingga setiap kategori dapat dianalisis secara komputasional pada tahap selanjutnya*

In [ ]:
df_encoded.head()

,Choco Oreo,Chocolate latte,Kopi Susu,Kopi gula aren,Mango yakult,Matcha latte,Milk Tea,Taro latte,Thai Tea,Yakult drink
0,True,False,True,True,False,True,False,False,True,False
1,False,False,True,True,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,True
3,False,True,True,False,False,False,False,False,False,True
4,False,False,True,False,False,False,False,False,False,True


In [ ]:
(df_encoded == 1).sum()

,0
Choco Oreo,8
Chocolate latte,4
Kopi Susu,6
Kopi gula aren,6
Mango yakult,4
Matcha latte,5
Milk Tea,3
Taro latte,3
Thai Tea,4
Yakult drink,5


# **Apriori**

In [ ]:
#Apriori
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.1,
    use_colnames=True
    )

#generate rules
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.5
  )

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                       antecedents                   consequents  support  \
0                 (Kopi gula aren)                  (Choco Oreo)     0.15   
1                   (Mango yakult)                  (Choco Oreo)     0.10   
2                   (Matcha latte)                  (Choco Oreo)     0.20   
3                     (Choco Oreo)                (Matcha latte)     0.20   
4                       (Milk Tea)             (Chocolate latte)     0.10   
5                (Chocolate latte)                    (Milk Tea)     0.10   
6                 (Kopi gula aren)                   (Kopi Susu)     0.20   
7                      (Kopi Susu)              (Kopi gula aren)     0.20   
8                       (Thai Tea)                   (Kopi Susu)     0.10   
9                 (Kopi gula aren)                (Matcha latte)     0.15   
10                  (Matcha latte)              (Kopi gula aren)     0.15   
11                      (Thai Tea)              (Kopi gula aren)     0.10   

*Pada pengujian ini, minimum support 0,1 menunjukkan itemset dianggap frequent jika muncul pada minimal 10% transaksi. Dengan minimum confidence 0,5, aturan diterima jika kepercayaan mencapai 50% atau lebih. Pengaturan ini menghasilkan banyak aturan yang kompleks*

In [ ]:
#Apriori
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.2,
    use_colnames=True
    )

#generate rules
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.5
  )

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

        antecedents       consequents  support  confidence      lift
0    (Matcha latte)      (Choco Oreo)      0.2    0.800000  2.000000
1      (Choco Oreo)    (Matcha latte)      0.2    0.500000  2.000000
2  (Kopi gula aren)       (Kopi Susu)      0.2    0.666667  2.222222
3       (Kopi Susu)  (Kopi gula aren)      0.2    0.666667  2.222222


*Pada pengujian ini, minimum support ditetapkan 0,2, artinya aturan yang muncul hanya untuk itemset yang muncul minimal 20% dari seluruh transaksi. Dengan minimum confidence 0,5, aturan diterima jika kepercayaan ≥50%. Hasil menunjukkan hanya dua aturan yang muncul: "Kopi gula aren → Kopi Susu" dan sebaliknya, keduanya memiliki confidence 66,7% dan lift 2,22. Hal ini menunjukkan hubungan yang cukup kuat antar item, namun jumlah aturan lebih sedikit dibandingkan pengaturan support yang lebih rendah*

In [ ]:
#Apriori
frequent_itemsets = apriori(
    df_encoded,
    min_support=0.1,
    use_colnames=True
    )

#generate rules
rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.7
  )

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                      antecedents       consequents  support  confidence  \
0                  (Matcha latte)      (Choco Oreo)     0.20        0.80   
1  (Kopi gula aren, Matcha latte)      (Choco Oreo)     0.15        1.00   
2    (Kopi gula aren, Choco Oreo)    (Matcha latte)     0.15        1.00   
3      (Matcha latte, Choco Oreo)  (Kopi gula aren)     0.15        0.75   
4      (Kopi gula aren, Thai Tea)       (Kopi Susu)     0.10        1.00   
5           (Kopi Susu, Thai Tea)  (Kopi gula aren)     0.10        1.00   

       lift  
0  2.000000  
1  2.500000  
2  4.000000  
3  2.500000  
4  3.333333  
5  3.333333  


In [ ]:
from itertools import combinations

# List semua item
items = df_encoded.columns.tolist()

# Generate semua kombinasi 2-item
C2 = list(combinations(items, 2))
print("Candidate 2-itemsets (C2):")
print(C2)

Candidate 2-itemsets (C2):
[('Choco Oreo', 'Chocolate latte'), ('Choco Oreo', 'Kopi Susu'), ('Choco Oreo', 'Kopi gula aren'), ('Choco Oreo', 'Mango yakult'), ('Choco Oreo', 'Matcha latte'), ('Choco Oreo', 'Milk Tea'), ('Choco Oreo', 'Taro latte'), ('Choco Oreo', 'Thai Tea'), ('Choco Oreo', 'Yakult drink'), ('Chocolate latte', 'Kopi Susu'), ('Chocolate latte', 'Kopi gula aren'), ('Chocolate latte', 'Mango yakult'), ('Chocolate latte', 'Matcha latte'), ('Chocolate latte', 'Milk Tea'), ('Chocolate latte', 'Taro latte'), ('Chocolate latte', 'Thai Tea'), ('Chocolate latte', 'Yakult drink'), ('Kopi Susu', 'Kopi gula aren'), ('Kopi Susu', 'Mango yakult'), ('Kopi Susu', 'Matcha latte'), ('Kopi Susu', 'Milk Tea'), ('Kopi Susu', 'Taro latte'), ('Kopi Susu', 'Thai Tea'), ('Kopi Susu', 'Yakult drink'), ('Kopi gula aren', 'Mango yakult'), ('Kopi gula aren', 'Matcha latte'), ('Kopi gula aren', 'Milk Tea'), ('Kopi gula aren', 'Taro latte'), ('Kopi gula aren', 'Thai Tea'), ('Kopi gula aren', 'Yakult d

In [ ]:
# Minimum support threshold, misal 0.1 (10%)
min_support = 0.1

L2 = []  # list untuk menyimpan frequent 2-itemsets
support_L2 = {}

for itemset in C2:
    # Hitung support = jumlah transaksi yang mengandung kedua item / total transaksi
    support = df_encoded[list(itemset)].all(axis=1).sum() / len(df_encoded)
    if support >= min_support:
        L2.append(itemset)
        support_L2[itemset] = support

print("\nFrequent 2-itemsets (L2) dengan support >= 0.1:")
for itemset in L2:
    print(f"{itemset} - support: {support_L2[itemset]:.2f}")


Frequent 2-itemsets (L2) dengan support >= 0.1:
('Choco Oreo', 'Kopi gula aren') - support: 0.15
('Choco Oreo', 'Mango yakult') - support: 0.10
('Choco Oreo', 'Matcha latte') - support: 0.20
('Chocolate latte', 'Milk Tea') - support: 0.10
('Kopi Susu', 'Kopi gula aren') - support: 0.20
('Kopi Susu', 'Thai Tea') - support: 0.10
('Kopi Susu', 'Yakult drink') - support: 0.10
('Kopi gula aren', 'Matcha latte') - support: 0.15
('Kopi gula aren', 'Thai Tea') - support: 0.10
('Mango yakult', 'Matcha latte') - support: 0.10
('Mango yakult', 'Milk Tea') - support: 0.10


In [ ]:
total_transactions = len(df_encoded)

print("\nFrequency (count) of Frequent 2-itemsets (L2):")
for itemset in L2:
    frequency = support_L2[itemset] * total_transactions
    print(f"{itemset} - frequency: {int(frequency)}")


Frequency (count) of Frequent 2-itemsets (L2):
('Choco Oreo', 'Kopi gula aren') - frequency: 3
('Choco Oreo', 'Mango yakult') - frequency: 2
('Choco Oreo', 'Matcha latte') - frequency: 4
('Chocolate latte', 'Milk Tea') - frequency: 2
('Kopi Susu', 'Kopi gula aren') - frequency: 4
('Kopi Susu', 'Thai Tea') - frequency: 2
('Kopi Susu', 'Yakult drink') - frequency: 2
('Kopi gula aren', 'Matcha latte') - frequency: 3
('Kopi gula aren', 'Thai Tea') - frequency: 2
('Mango yakult', 'Matcha latte') - frequency: 2
('Mango yakult', 'Milk Tea') - frequency: 2
